In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
filepath = '../Dataset/'
train = pd.read_csv(filepath + 'Train.csv')
test = pd.read_csv(filepath + 'Test.csv')
dataDic = pd.read_csv(filepath + 'DataDictionary.csv')

In [3]:
train.shape, test.shape, dataDic.shape

((44907, 45), (1528, 6), (44, 2))

The train data has 44907 entries which is to be used to make a prediction on 1528 entries.

Interestingly, the test data has just 6 rows of data compared to the 45 rows of data in the train data.

A deeper look is needed especially with the fact that there are 44 described features in the Data Dictionary

In [4]:
test.head()

,sku_name,month,year,CAT_GENDER_BOTH,CAT_GENDER_MEN,CAT_GENDER_WOMEN
0,YOSHTLYNYOSHZZ,11,2021,0,1,0
1,YOSHTLYNYOSHZZ,12,2021,0,1,0
2,YOSHTLYNYOSHZZ,1,2022,0,1,0
3,YOSHTLYNYOSHZZ,2,2022,0,1,0
4,YOSHRENECARL,11,2021,0,0,1


In [5]:
test.columns

Index(['sku_name', 'month', 'year', 'CAT_GENDER_BOTH', 'CAT_GENDER_MEN',
       'CAT_GENDER_WOMEN'],
      dtype='object')

In [6]:
train.head()

,sku_name,starting_inventory,sellin,sellin_channel_1,sellin_channel_2,sellin_channel_3,sellin_channel_4,sellin_channel_5,sellin_channel_6,sellin_channel_7,...,month,year,product_lifecycle_stage,FLAG100,disc_month,cum_disc,CAT_GENDER_BOTH,CAT_GENDER_MEN,CAT_GENDER_WOMEN,Weeks
0,YOSHWARDTERR,0,1013,0,0,0,1013,0,0,0,...,7,2016,U,0.000000,0,0,0,1,0,1
1,YOSHWARDTERR,0,2026,0,0,0,2026,0,0,0,...,2,2017,U,0.000000,0,0,0,1,0,0
2,YOSHWARDTERR,0,1013,0,0,0,1013,0,0,0,...,5,2017,U,0.000000,0,0,0,1,0,0
3,YOSHUANEMARX,0,320108,4052,40520,240081,4052,6078,18234,0,...,4,2018,W,0.270966,1,1,0,0,1,1
4,YOSHUANEMARX,0,132703,2026,0,81040,3039,25325,18234,0,...,5,2018,W,0.063004,0,1,0,0,1,0


In [7]:
train.columns

Index(['sku_name', 'starting_inventory', 'sellin', 'sellin_channel_1',
       'sellin_channel_2', 'sellin_channel_3', 'sellin_channel_4',
       'sellin_channel_5', 'sellin_channel_6', 'sellin_channel_7',
       'sellin_channel_8', 'sellout', 'onhand_inventory', 'leftover_inventory',
       'sellout_channel_1', 'sellout_channel_2', 'sellout_channel_3',
       'sellout_channel_4', 'sellout_channel_5', 'sellout_channel_6',
       'sellout_channel_7', 'sellout_channel_8', 'sellout_channel_9',
       'sellout_channel_10', 'onhand_inventory_channel_1',
       'onhand_inventory_channel_2', 'onhand_inventory_channel_3',
       'onhand_inventory_channel_4', 'onhand_inventory_channel_5',
       'onhand_inventory_channel_6', 'onhand_inventory_channel_7',
       'onhand_inventory_channel_8', 'onhand_inventory_channel_9',
       'onhand_inventory_channel_10', 'price', 'month', 'year',
       'product_lifecycle_stage', 'FLAG100', 'disc_month', 'cum_disc',
       'CAT_GENDER_BOTH', 'CAT_GENDER_MEN',

In [8]:
dataDic.head()

,Column Name,Description
0,sku_name,Product SKU
1,starting_inventory,Starting inventory at the beginning of the mon...
2,sellin,The sellin or demand. What is sold in to our c...
3,sellin_channel_1,The sellin for channel 1
4,sellin_channel_2,The sellin for channel 2


In [9]:
dataDic.tail()

,Column Name,Description
39,cum_disc,The effect of a discount may not be captured b...
40,CAT_GENDER_BOTH,SKU is male and female
41,CAT_GENDER_MEN,SKU is male
42,CAT_GENDER_WOMEN,SKU is female
43,Weeks,The number of weekends that certain month had


"""
The objective is to accurately predict the “sellin”, which is the optimal inventory needed to meet demand by Fossil to its own customers or channels, four months into the future.
"""

According to the task description

In [10]:
train.sellin

0          1013
1          2026
2          1013
3        320108
4        132703
          ...  
44902    156002
44903     34442
44904     38494
44905     29377
44906     18234
Name: sellin, Length: 44907, dtype: int64

In [11]:
dataDic[dataDic['Column Name'] == 'sellin'].iloc[0,1]

'The sellin or demand. What is sold in to our customers. \nThis is what we are trying to predict 4 months from now (Dependent)'

We are trying to predict the above

In [12]:
test_col = list(test.columns)

train[test_col].head()

,sku_name,month,year,CAT_GENDER_BOTH,CAT_GENDER_MEN,CAT_GENDER_WOMEN
0,YOSHWARDTERR,7,2016,0,1,0
1,YOSHWARDTERR,2,2017,0,1,0
2,YOSHWARDTERR,5,2017,0,1,0
3,YOSHUANEMARX,4,2018,0,0,1
4,YOSHUANEMARX,5,2018,0,0,1


A baseline model is to build use the above only in making the prediction

In [13]:
train[test_col].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44907 entries, 0 to 44906
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   sku_name          44907 non-null  object
 1   month             44907 non-null  int64 
 2   year              44907 non-null  int64 
 3   CAT_GENDER_BOTH   44907 non-null  int64 
 4   CAT_GENDER_MEN    44907 non-null  int64 
 5   CAT_GENDER_WOMEN  44907 non-null  int64 
dtypes: int64(5), object(1)
memory usage: 2.1+ MB


In [14]:
train[test_col].describe()

,month,year,CAT_GENDER_BOTH,CAT_GENDER_MEN,CAT_GENDER_WOMEN
count,44907.000000,44907.000000,44907.000000,44907.000000,44907.000000
mean,6.495068,2017.872759,0.012648,0.448037,0.539315
std,3.356170,1.626007,0.111753,0.497298,0.498458
min,1.000000,2016.000000,0.000000,0.000000,0.000000
25%,4.000000,2016.000000,0.000000,0.000000,0.000000
50%,6.000000,2018.000000,0.000000,0.000000,1.000000
75%,10.000000,2019.000000,0.000000,1.000000,1.000000
max,12.000000,2021.000000,1.000000,1.000000,1.000000


In [15]:
train.sku_name.nunique()

3868

In [37]:
X = train[test_col]
y = train['sellin']

train_var = {}
def base_processing(data):
    data = data.drop('sku_name',axis=1)
    return data

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(base_processing(X),y,test_size=0.2)

In [17]:
from sklearn.ensemble import RandomForestRegressor as RFR

reg1 = RFR(random_state=74)

reg1.fit(X_train,y_train)

RandomForestRegressor(random_state=74)

In [18]:
y_pred = reg1.predict(X_test)

from sklearn.metrics import mean_absolute_error as mae_

mae_(y_pred,y_test)

310330.38831812906

In [19]:
predictions = reg1.predict(base_processing(test))

In [20]:
test_copy = test.copy()
test_copy['ID'] = test_copy.sku_name+'_'+test_copy.month.astype('string')+'_'+test_copy.year.astype('string')

In [21]:
result = pd.DataFrame({
    'Item_ID':test_copy.ID,
    'Target':predictions
})
result=result.set_index('Item_ID')

result.to_csv('submission_quick_01.csv')


According to the submission, this model scores an MAE of 292377.2029346923 which took the 51st position on the leaderboard at the time of submission.

Not too Bad for a start!

### Looking at the other features

In [22]:
X.head()

,sku_name,month,year,CAT_GENDER_BOTH,CAT_GENDER_MEN,CAT_GENDER_WOMEN
0,YOSHWARDTERR,7,2016,0,1,0
1,YOSHWARDTERR,2,2017,0,1,0
2,YOSHWARDTERR,5,2017,0,1,0
3,YOSHUANEMARX,4,2018,0,0,1
4,YOSHUANEMARX,5,2018,0,0,1


In [23]:
for i in range(dataDic.shape[0]):
    print(dataDic.iloc[i,0],dataDic.iloc[i,1],sep="\t-\t")

sku_name	-	Product SKU
starting_inventory	-	Starting inventory at the beginning of the month that is 
at the dome and unclaimed by a retailer
sellin	-	The sellin or demand. What is sold in to our customers. 
This is what we are trying to predict 4 months from now (Dependent)
sellin_channel_1	-	The sellin for channel 1
sellin_channel_2	-	The sellin for channel 2
sellin_channel_3	-	The sellin for channel 3
sellin_channel_4	-	The sellin for channel 4
sellin_channel_5	-	The sellin for channel 5
sellin_channel_6	-	The sellin for channel 6
sellin_channel_7	-	The sellin for channel 7
sellin_channel_8	-	The sellin for channel 8
sellout	-	What is sold to the final customer
onhand_inventory	-	Onhand customer side inventory
sellout_channel_1	-	The sellout for channel 1
sellout_channel_2	-	The sellout for channel 2
sellout_channel_3	-	The sellout for channel 3
sellout_channel_4	-	The sellout for channel 4
sellout_channel_5	-	The sellout for channel 5
sellout_channel_6	-	The sellout for channel 6
s

In [24]:
X.sort_values(['year','month'])

,sku_name,month,year,CAT_GENDER_BOTH,CAT_GENDER_MEN,CAT_GENDER_WOMEN
50,YOSHTLYNYOSH,1,2016,0,1,0
69,YOSHRTHATRAN,1,2016,0,0,1
139,YOSHOVANTERR,1,2016,0,0,1
163,YOSHLROYARTI,1,2016,0,0,1
256,YOSHLEENARMA,1,2016,0,1,0
...,...,...,...,...,...,...
44546,ABERTHAKEVAZZ,10,2021,0,0,1
44708,ABEETTEABE,10,2021,0,1,0
44773,ABEENNEARMAZZ,10,2021,0,0,1
44807,ABEANNAONEIZZ,10,2021,0,0,1


In [25]:
X['sku_name'].str[:3].nunique()

91

In [28]:
X.head()

,sku_name,month,year,CAT_GENDER_BOTH,CAT_GENDER_MEN,CAT_GENDER_WOMEN
0,YOSHWARDTERR,7,2016,0,1,0
1,YOSHWARDTERR,2,2017,0,1,0
2,YOSHWARDTERR,5,2017,0,1,0
3,YOSHUANEMARX,4,2018,0,0,1
4,YOSHUANEMARX,5,2018,0,0,1


In [38]:
def base_processing2(data, data_type='train',first_n=2):
    data = data.copy()
    data[f'sku_first{first_n}'] = data['sku_name'].str[:first_n]
    data = data.drop('sku_name',axis=1)
    
    if 'ID' in data.columns:
        data = data.drop('ID',axis=1)
    
    data = pd.get_dummies(data)
    
    if data_type=='test':
        for train_col in X_train.columns:
            if train_col not in data.columns:
                #print(train_col)
                data[train_col] = 0
        data = data[X_train.columns]
    
    return data

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(base_processing2(X),y,test_size=0.2)

In [41]:
X.head()

,sku_name,month,year,CAT_GENDER_BOTH,CAT_GENDER_MEN,CAT_GENDER_WOMEN
0,YOSHWARDTERR,7,2016,0,1,0
1,YOSHWARDTERR,2,2017,0,1,0
2,YOSHWARDTERR,5,2017,0,1,0
3,YOSHUANEMARX,4,2018,0,0,1
4,YOSHUANEMARX,5,2018,0,0,1


In [62]:
X_train.columns

Index(['month', 'year', 'CAT_GENDER_BOTH', 'CAT_GENDER_MEN',
       'CAT_GENDER_WOMEN', 'sku_first3_AB', 'sku_first3_AL', 'sku_first3_AM',
       'sku_first3_AN', 'sku_first3_AR',
       ...
       'sku_first4_SH', 'sku_first4_SI', 'sku_first4_SU', 'sku_first4_TE',
       'sku_first4_TH', 'sku_first4_TR', 'sku_first4_VI', 'sku_first4_WA',
       'sku_first4_WH', 'sku_first4_YO'],
      dtype='object', length=200)

In [63]:
from sklearn.ensemble import RandomForestRegressor as RFR

reg1 = RFR(random_state=74)

reg1.fit(X_train,y_train)

RandomForestRegressor(random_state=74)

The previous value is 311094.3248697206, 334687.9424594393, 332658.473115057, 
330203.23916765075, 327506.14753984445, 320387.90283087996

In [64]:
y_pred = reg1.predict(X_test)

from sklearn.metrics import mean_absolute_error as mae_

mae_(y_pred,y_test)

332658.473115057

In [34]:
base_processing2(test,data_type='test')

,month,year,CAT_GENDER_BOTH,CAT_GENDER_MEN,CAT_GENDER_WOMEN,sku_first3_ABE,sku_first3_ALB,sku_first3_ALO,sku_first3_AMB,sku_first3_ANH,...,sku_first3_SUN,sku_first3_TER,sku_first3_THO,sku_first3_TRA,sku_first3_WAL,sku_first3_WHI,sku_first3_YOS,sku_first3_BAR,sku_first3_HAN,sku_first3_VIC
0,11,2021,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,12,2021,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,1,2022,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,2,2022,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,11,2021,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1523,2,2022,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1524,11,2021,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1525,12,2021,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1526,1,2022,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
predictions = reg1.predict(base_processing2(test,data_type='test'))

In [36]:
test['ID'] = test.sku_name+'_'+test.month.astype('string')+'_'+test.year.astype('string')

In [37]:
result = pd.DataFrame({
    'Item_ID':test.ID,
    'Target':predictions
})
result=result.set_index('Item_ID')

result.to_csv('submission_quick_02.csv')
